In [30]:
import pandas as pd

inter = pd.read_csv('interactions.csv')
inter["datetime"] = pd.to_datetime(inter["last_watch_dt"])
inter.head(3)

,user_id,item_id,last_watch_dt,total_dur,watched_pct,datetime
0,176549,9506,2021-05-11,4250,72.0,2021-05-11
1,699317,1659,2021-05-29,8317,100.0,2021-05-29
2,656683,7107,2021-05-09,10,0.0,2021-05-09


In [32]:
inter['datetime'].min()

Timestamp('2021-03-13 00:00:00')

In [33]:
inter['datetime'].max()

Timestamp('2021-08-22 00:00:00')

In [36]:
inter[inter['datetime'] == '2021-08-19']

,user_id,item_id,last_watch_dt,total_dur,watched_pct,datetime
64,73446,14488,2021-08-19,6011,100.0,2021-08-19
115,678206,9169,2021-08-19,5958,100.0,2021-08-19
175,721680,4880,2021-08-19,7659,9.0,2021-08-19
291,335852,13290,2021-08-19,2436,44.0,2021-08-19
347,619155,11912,2021-08-19,7559,100.0,2021-08-19
...,...,...,...,...,...,...
5475286,684895,16284,2021-08-19,3137,60.0,2021-08-19
5475291,886900,512,2021-08-19,164122,100.0,2021-08-19
5475565,900949,15171,2021-08-19,178,3.0,2021-08-19
5475640,890278,12360,2021-08-19,684,8.0,2021-08-19


In [20]:
items_df = pd.read_csv('items.csv')
items_df.head(1)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."


In [18]:
items_df.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [ ]:
items_df = items_df[['realease_year', 'age_rating']]

In [9]:
import pandas as pd
from rectools.dataset import Dataset

users_df = pd.read_csv("users.csv")
if "user_id" not in users_df.columns and users_df.index.name is not None:
    users_df = users_df.reset_index().rename(columns={users_df.index.name: "user_id"})
if "user_id" not in users_df.columns:
    users_df = users_df.reset_index().rename(columns={"index":"user_id"})

items_df = pd.read_csv("items.csv")
if "item_id" not in items_df.columns and items_df.index.name is not None:
    items_df = items_df.reset_index().rename(columns={items_df.index.name: "item_id"})
if "item_id" not in items_df.columns:
    items_df = items_df.reset_index().rename(columns={"index":"item_id"})

users_df.head(3)
# inter_df = pd.read_csv("interactions.csv", parse_dates=["last_watch_dt"])
# inter_df = inter_df.rename(columns={"last_watch_dt":"datetime"})

# dataset = Dataset.construct(
#     interactions_df=inter_df.rename(columns={"user_id":"user_id","item_id":"item_id","datetime":"datetime","weight":"weight"}),
#     user_features_df=users_df,
#     item_features_df=items_df
# )


,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0


In [1]:
import pandas as pd
from rectools.dataset import Dataset
from rectools.models import PopularModel, EASEModel, ImplicitALSWrapperModel, ImplicitItemKNNWrapperModel, LightFMWrapperModel, PureSVDModel
from rectools.model_selection import LastNSplitter
from pathlib import Path
from rectools.model_selection import TimeRangeSplitter

INTERACTIONS_CSV = "interactions.csv"
USERS_CSV = "users.csv"
ITEMS_CSV = "items.csv"

inter_df = pd.read_csv(INTERACTIONS_CSV, parse_dates=["last_watch_dt"])
inter_df = inter_df.rename(columns={"last_watch_dt":"datetime"})
inter_df['weight'] = inter_df['watched_pct']
users_path = Path(USERS_CSV)
if users_path.exists():
    users_df = pd.read_csv(users_path)
    users_df = users_df[['user_id', 'kids_flg']]
    if "user_id" not in users_df.columns:
        users_df = users_df.reset_index().rename(columns={users_df.index.name or "index":"user_id"})
    if set(["feature","value"]).issubset(set(users_df.columns)):
        user_feats = users_df[["user_id","feature","value"]]
    else:
        value_vars = [c for c in users_df.columns if c != "user_id"]
        user_feats = users_df.melt(id_vars=["user_id"], value_vars=value_vars, var_name="feature", value_name="value")
        user_feats = user_feats.dropna(subset=["value"])
else:
    user_feats = None

items_path = Path(ITEMS_CSV)
if items_path.exists():
    items_df = pd.read_csv(items_path)
    items_df = items_df[['release_year', 'age_rating']]
    if "item_id" not in items_df.columns:
        items_df = items_df.reset_index().rename(columns={items_df.index.name or "index":"item_id"})
    if set(["feature","value"]).issubset(set(items_df.columns)):
        item_feats = items_df[["item_id","feature","value"]]
    else:
        value_vars = [c for c in items_df.columns if c != "item_id"]
        item_feats = items_df.melt(id_vars=["item_id"], value_vars=value_vars, var_name="feature", value_name="value")
        item_feats = item_feats.dropna(subset=["value"])
else:
    item_feats = None

dataset = Dataset.construct(
    interactions_df=inter_df.rename(columns={"user_id":"user_id","item_id":"item_id","datetime":"datetime","weight":"weight"}),
    user_features_df=user_feats,
    item_features_df=item_feats
)

c:\Users\ЯРОСЛАВ\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [6]:
import pandas as pd
from rectools.dataset import Dataset
from rectools.model_selection import LastNSplitter
from rectools.models import PopularModel, EASEModel, ImplicitALSWrapperModel, ImplicitItemKNNWrapperModel, LightFMWrapperModel, PureSVDModel
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from lightfm import LightFM
from pathlib import Path


train_dataset = dataset

als_base = AlternatingLeastSquares(factors=64, regularization=0.01, iterations=15)
itemknn_base = ItemItemRecommender(K=40)
lightfm_base = LightFM(no_components=64, loss='warp', random_state=42)


models = {
    "popular": PopularModel(),
    "ease": EASEModel(),
    "als": ImplicitALSWrapperModel(model=als_base),
    "itemknn": ImplicitItemKNNWrapperModel(model=itemknn_base),
    "lightfm": LightFMWrapperModel(model=lightfm_base),
    "pure_svd": PureSVDModel()
}


c:\Users\ЯРОСЛАВ\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [11]:
from tqdm.auto import tqdm
import time
OUTPUT_DIR = 'fun'
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

for name, model in tqdm(list(models.items()), desc="Models"):
    tqdm.write(f"Start training: {name}")
    t0 = time.time()
    model.fit(train_dataset)
    t1 = time.time()
    tqdm.write(f"Finished {name} in {t1 - t0:.1f}s")

Models:   0%|          | 0/6 [00:00<?, ?it/s]

Start training: popular
Finished popular in 10.8s
Start training: ease


In [ ]:
from tqdm.auto import trange

# предположим, что user_item_csr — scipy.sparse.csr_matrix (shape: n_users x n_items)
# и что als_base, lightfm_base — объекты библиотек implicit / lightfm
EPOCHS_ALS = 20
for epoch in trange(EPOCHS_ALS, desc="ALS epochs"):
    als_base.fit(user_item_csr, iterations=1)  # implicit ALS обычно продолжает обучение, если вызвать несколько раз
# затем упаковать в wrapper (если нужно) и использовать в rectools

EPOCHS_LF = 30
for epoch in trange(EPOCHS_LF, desc="LightFM epochs"):
    lightfm_base.fit(user_item_csr, epochs=1, num_threads=4)


In [ ]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

TEST_CSV = "test.csv"
USER_COL = "user_id"
TOP_K = 20
OUTPUT_DIR = "rectools_test_recs"

test_df = pd.read_csv(TEST_CSV)

if "dataset" in globals():
    users = test_df[USER_COL].astype(dataset.interactions.user_id.dtype).unique().tolist()
else:
    users = test_df[USER_COL].astype(str).unique().tolist()

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

model_recs = []
for name, model in tqdm(list(models.items()), desc="Models"):
    recs = model.recommend(users=users, dataset=dataset, k=TOP_K, filter_viewed=True)
    recs_df = pd.DataFrame(recs)
    recs_df.columns = ["user_id", "item_id", "score"]
    recs_df["model"] = name
    recs_df.to_csv(Path(OUTPUT_DIR) / f"recs_{name}.csv", index=False)
    model_recs.append(recs_df[["user_id", "item_id", "score"]])

ensemble_df = pd.concat(model_recs, ignore_index=True)
ensemble_mean = ensemble_df.groupby(["user_id", "item_id"], as_index=False)["score"].mean()
ensemble_sorted = ensemble_mean.sort_values(["user_id", "score"], ascending=[True, False])
ensemble_topk = ensemble_sorted.groupby("user_id").head(TOP_K)
ensemble_topk.to_csv(Path(OUTPUT_DIR) / "recs_ensemble_mean.csv", index=False)
